In [2]:
import logging
import os
import json
import argparse
import time
DATA_FOLDER = "data"
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

data_path = os.path.join(DATA_FOLDER, "sample.json")
save_rate = 10
break_time = 1000
mail = "nguyendichnhatminh@gmail.com"
password = "nguyendichnhatminh"
folder_path = "nguyendichnhatminh"


In [3]:

with open(os.path.join(DATA_FOLDER, "sample.json"), encoding="utf8") as json_file:
    qa_data = json.load(json_file)

qa_data

[{'question': 'Việc trích lập dự phòng nghiệp vụ của doanh nghiệp kinh doanh bảo hiểm phải bảo đảm những yêu cầu nào?',
  'passage': 'Khoản 2 Điều 97 Luật Kinh doanh bảo hiểm 2022 quy định về các yêu cầu với việc trích lập dự phòng nghiệp vụ như sau: Dự phòng nghiệp vụ 1. Dự phòng nghiệp vụ là khoản tiền mà doanh nghiệp bảo hiểm, doanh nghiệp tái bảo hiểm, chi nhánh nước ngoài tại Việt Nam phải trích lập nhằm mục đích thanh toán cho những trách nhiệm bảo hiểm có thể phát sinh từ các hợp đồng bảo hiểm đã giao kết. 2. Việc trích lập dự phòng nghiệp vụ phải bảo đảm các yêu cầu sau đây: a) Trích lập riêng cho từng nghiệp vụ bảo hiểm; b) Tương ứng với phần trách nhiệm đã cam kết theo thỏa thuận trong hợp đồng bảo hiểm; c) Tách biệt giữa các hợp đồng bảo hiểm của đối tượng bảo hiểm trong và ngoài phạm vi lãnh thổ Việt Nam, kể cả trong cùng một nghiệp vụ bảo hiểm, sản phẩm bảo hiểm, trừ trường hợp pháp luật có quy định khác; d) Luôn có tài sản tương ứng với dự phòng nghiệp vụ đã trích lập, đồ

In [4]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from fake_useragent import UserAgent
from selenium.common.exceptions import WebDriverException
import shutil
import copy


def openai_login(driver, email, password):

    curent_url = driver.current_url
    if "login" not in curent_url:
        driver.get('https://chat.openai.com/auth/login')

    inputElement = driver.find_element(
        By.XPATH, "//button[@data-testid='login-button']")
    inputElement.click()
    time.sleep(3)
    mail = driver.find_element(By.XPATH, "//input[@inputmode='email']")
    mail.send_keys(u'\ue009' + u'\ue003')
    mail.send_keys(email)
    btn = driver.find_elements(By.TAG_NAME, "button")[0]
    btn.click()
    time.sleep(0.75)
    password_element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//input[@id='password']"))
    )
    password_element.send_keys(password)
    btn = driver.find_element(
        By.XPATH, "//button[starts-with(text(),'Continue') and @data-action-button-primary]")
    btn.click()
    logging.info(f"Logging as {email} ...")

def skip_popups(driver):
    time.sleep(3)
    try:
        skip = driver.find_element(
            By.XPATH, "//div[starts-with(text(),'Okay, let')]")
        skip.click()
    except NoSuchElementException:
        logging.info("* Skipping skip_popups")

In [5]:
def get_driver_with_account(account, password, data_user_folder):

    op = uc.ChromeOptions()
    # op.add_argument("--headless")
    op.add_argument('--no-sandbox')
    op.add_argument('--disable-dev-shm-usage')
    op.add_argument(f"--user-agent={UserAgent.random}")
    op.add_argument(f"--user-data-dir=selenium-data/{data_user_folder}/")
    op_copy = copy.deepcopy(op)

    try:
        driver = uc.Chrome(options=op, use_subprocess=True)
    except WebDriverException as e:
        logging.info("Unknown Bug " + str(e))
        logging.info(e.stacktrace)
        logging.info("Attempting to debug the exception")
        selenium_data_path = f"selenium-data/{data_user_folder}/"
        if os.path.exists(selenium_data_path):
            shutil.rmtree(selenium_data_path)
            logging.info(f"Delete folder {selenium_data_path}")
            driver = uc.Chrome(options=op_copy, use_subprocess=True)

        else:
            logging.info(
                f"There is not exist {selenium_data_path} to delete and debug.")
            raise (e)

    driver.get("https://chat.openai.com/")
    time.sleep(5)
    curent_url = driver.current_url
    if "login" in curent_url:
        try:
            openai_login(driver, account, password)
            time.sleep(10)
        except Exception as e:
            logging.info("--------------------------------")
            logging.info(f"Something wrong when logging as {account}.")
            logging.info("--------------------------------")
            raise (e)

    skip_popups(driver)
    return driver


In [19]:

from SeleniumChatGPT import get_response
from newTvlpOneAccount import get_driver_with_account

driver = get_driver_with_account(mail, password, folder_path)

2023-10-23 13:07:57 - * Skipping skip_popups


In [12]:
if "oke" in  qa_data[0].keys():
    print("hehe")
else:
    print("haha")

hehe


In [20]:
def generate_feeds(driver, input_path, save_rate=1000):

    # Read normal jsonl data
    with open(input_path, encoding="utf8") as json_file:
        qa_data = json.load(json_file)

    for idx, qa in enumerate(qa_data):
        if "oke" in qa.keys():
            if qa["oke"] == "True" or qa["oke"] == "False":
                continue
        
        passage = qa["passage"]
        question = qa["question"]
        prompt = f"""

You are a Vietnamese , you can understand anything in Vietnamese. You are also a good researcher and you have ability to read a passage, think step by step and decide whether the information in the passage can be use to find the answer for any questions.


Cho ngữ cảnh sau

'{passage}'

Liệu ngữ cảnh trên có thể dùng để trả lời câu hỏi '{question}' không? Hãy giải thích chi tiết và trả lời ở CUỐI CÂU là 'True' nếu có và 'False' nếu không, lưu ý 'True' và 'False' phải được đặt ở cuối câu.

OUTPUT:
"""        # prompt_preprocess = prompt.replace("\n", Keys.chord(Keys.SHIFT, Keys.ENTER));
        try:
            response = get_response(
                driver, prompt=prompt, waittime=100)
            qa["oke"] = response.strip()
            logging.info(
                f"Question ID {idx}  - {response}")

        except Exception as e:
            raise (e)
        # if (idx + 1) % save_rate == 0:
        #     with open(input_path, "w", encoding="utf8") as file:
        #         json.dump(qa_data, file)
        #         logging.info("Saving....")

    # with open(input_path, "w", encoding="utf8") as file:
    #     json.dump(qa_data, file)
    #     logging.info("Saving....")
    logging.info("Finish.")
    return qa_data

In [21]:
prompt = f"""
You are a Vietnamese , you can understand anything in Vietnamese. You are also a good researcher and you have ability to read a passage and decide whether the information in the passage can be use to find the answer for any questions, if yes you only answer 'True' and if not you only answer 'False' and hate explaining
Cho ngữ cảnh sau

'{password}'

Liệu ngữ cảnh trên có thể dùng để trả lời câu hỏi '{mail}' không, nhấn mạnh là chỉ sử dụng thông tin từ ngữ cảnh trên và không sử dụng thêm bất cứ thông tin bên ngoài nào?. Nếu có hãy trả lời duy nhất một từ là 'True' còn nếu không trả lời duy nhất một từ 'False' và không cần giải thích

OUTPUT:
"""

In [25]:
'True' in "Cau tra loi la 'True'"

True

In [22]:
print(prompt)


You are a Vietnamese , you can understand anything in Vietnamese. You are also a good researcher and you have ability to read a passage and decide whether the information in the passage can be use to find the answer for any questions, if yes you only answer 'True' and if not you only answer 'False' and hate explaining
Cho ngữ cảnh sau

'nguyendichnhatminh'

Liệu ngữ cảnh trên có thể dùng để trả lời câu hỏi 'nguyendichnhatminh@gmail.com' không, nhấn mạnh là chỉ sử dụng thông tin từ ngữ cảnh trên và không sử dụng thêm bất cứ thông tin bên ngoài nào?. Nếu có hãy trả lời duy nhất một từ là 'True' còn nếu không trả lời duy nhất một từ 'False' và không cần giải thích

OUTPUT:



In [23]:
generated = generate_feeds(driver=driver,input_path=os.path.join(DATA_FOLDER, "sample_generated.json"))

2023-10-23 13:08:39 - Question ID 0  - Có, ngữ cảnh trên cung cấp thông tin chi tiết về các yêu cầu cần phải bảo đảm khi trích lập dự phòng nghiệp vụ của doanh nghiệp kinh doanh bảo hiểm theo Điều 97 Luật Kinh doanh bảo hiểm 2022. Cụ thể, các yêu cầu đó bao gồm:
a) Trích lập riêng cho từng nghiệp vụ bảo hiểm.
b) Tương ứng với phần trách nhiệm đã cam kết theo thỏa thuận trong hợp đồng bảo hiểm.
c) Tách biệt giữa các hợp đồng bảo hiểm của đối tượng bảo hiểm trong và ngoài phạm vi lãnh thổ Việt Nam, kể cả trong cùng một nghiệp vụ bảo hiểm, trừ trường hợp pháp luật có quy định khác.
d) Luôn có tài sản tương ứng với dự phòng nghiệp vụ đã trích lập, đồng thời tách biệt tài sản tương ứng với dự phòng quy định.
đ) Sử dụng Chuyên gia tính toán để tính toán, trích lập dự phòng nghiệp vụ.
e) Thường xuyên rà soát, đánh giá việc trích lập dự phòng nghiệp vụ; kịp thời có các biện pháp nhằm bảo đảm trích lập đầy đủ dự phòng để chi trả cho các trách nhiệm của doanh nghiệp bảo hiểm, doanh nghiệp tái bả

In [14]:
generated[7]

{'question': 'Biển cấm đỗ xe theo giờ trong khu vực phải được sử dụng kiểu chữ như thế nào?  ',
 'context': ' Tại Mục D.19 Phụ lục D QCVN 41:2019/BGTVT về báo hiệu đường bộ quy định: Biển hiệu lệnh có tác dụng trong khu vực Để báo cấm, hạn chế hay chỉ dẫn có hiệu lực cho tất cả các tuyến đường trong một khu vực (hiệu lực cho cả khu vực), đặt biển “Bắt đầu vào khu vực” (Ví dụ: biển số R.E,9a; R.E,9b; R.E,9c; R.E,9d; ...). Từ ZONE được biểu thị ở phía trên và chi tiết cấm, hạn chế hay chỉ dẫn được biểu thị ở bên dưới. Tùy theo chi tiết cấm, hạn chế hay chỉ dẫn mà bố trí biển tương ứng, ví dụ: - Để cấm đỗ xe trong khu vực, đặt biển R.E,9a “Cấm đỗ xe trong khu vực”; - Để cấm đỗ xe theo giờ trong khu vực, đặt biển R.E,9b “Cấm đỗ xe theo giờ trong khu vực”; - Để quy định khu vực đỗ xe, đặt biển R.E,9c “Khu vực đỗ xe”; - Để quy định hạn chế tốc độ tối đa trong khu vực, đặt biển R.E,9d “Hạn chế tốc độ tối đa trong khu vực”.  Theo quy định tại Mục D.20 Phụ lục D QCVN 41:2019/BGTVT quy định: Biể

In [45]:
generated[7]

{'question': 'Tốt nghiệp cao đẳng có được tuyển chọn đào tạo sĩ quan dự bị không?',
 'passage': 'Căn cứ Điều 4 Nghị định 78/2020/NĐ-CP quy định như sau: Đối tượng, tiêu chuẩn tuyển chọn đào tạo sĩ quan dự bị 1. Đối tượng tuyển chọn a) Quân nhân chuyên nghiệp, hạ sĩ quan chuẩn bị xuất ngũ; b) Hạ sĩ quan dự bị, cán bộ, công chức, viên chức và công dân tốt nghiệp đại học trở lên; c) Sinh viên khi tốt nghiệp đại học. 2. Tiêu chuẩn tuyển chọn a) Tiêu chuẩn chung Đối tượng quy định tại khoản 1 Điều này có lịch sử chính trị của gia đình và bản thân rõ ràng, là đảng viên Đảng Cộng sản Việt Nam hoặc có đủ tiêu chuẩn chính trị để bồi dưỡng phát triển thành đảng viên Đảng Cộng sản Việt Nam; có bản lĩnh chính trị vững vàng, phẩm chất đạo đức tốt, tuyệt đối trung thành với Đảng, với Tổ quốc và Nhân dân; có trình độ, kiến thức chuyên môn, học vấn, tuổi đời phù hợp với từng đối tượng; sức khỏe từ loại 01 đến loại 03 theo quy định tuyển chọn sức khỏe tuyển quân, tuyển sinh quân sự của Bộ trưởng Bộ Y t